In [4]:
import pandas as pd
import os

In [5]:
train_path = "split_data/train.csv"
test_path = "split_data/test.csv"

df_train = pd.read_csv(train_path)
df_train.sample(5)

df_test = pd.read_csv(test_path)
df_test.sample(1)

,natural_language_query,sql_query
277,i also need service from dallas to boston arri...,SELECT DISTINCT flight_1.flight_id FROM flight...


In [6]:
df_prompt = df_train.sample(3)
df_test_in = df_test.sample(1)

In [7]:
input_str = ""
for i in range(len(df_prompt)):
    input_str += f"[EXAMPLE NL QUESTION {i+1}] "
    input_str += df_prompt.iloc[i]["natural_language_query"]
    input_str += "\n"
    input_str += f"[EXAMPLE SQL PROMPT {i+1}] "
    input_str += df_prompt.iloc[i]["sql_query"]
    input_str += "\n\n"

input_str += f"[NL QUESTION] "
input_str += df_test_in.iloc[0]["natural_language_query"]
input_str += f"\n[SQL PROMPT] "
input_str = """
You are a strict SQL translator. \n
You MUST ONLY output a single SQL query. \n
You are NOT allowed to repeat the instructions, the examples, or any natural language text. \n
You CANNOT explain anything. \n
You MUST copy the style of the EXAMPLES exactly.

Respond ONLY with the SQL query.
\n---\n
""" + input_str


print(input_str)


You are a strict SQL translator. 

You MUST ONLY output a single SQL query. 

You are NOT allowed to repeat the instructions, the examples, or any natural language text. 

You CANNOT explain anything. 

You MUST copy the style of the EXAMPLES exactly.

Respond ONLY with the SQL query.

---

[EXAMPLE NL QUESTION 1] i am interested in a flight on american airlines from dallas to washington
[EXAMPLE SQL PROMPT 1] SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 WHERE flight_1.airline_code = 'AA' AND ( flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DALLAS' AND flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'WASHINGTON' )

[EXAMPLE NL QUESTION 2] what transportation is available from the dallas airport to downtown
[EXAMPLE S

In [9]:
cur_dir = os.getcwd()

model_path = '../CHAT_MODEL/'
os.chdir(model_path)

from chat import main

os.chdir(cur_dir)

In [10]:
main(model_name="llama3.2:1b", prompt= input_str)

[WARNING] Could not fetch short summary: name 'model_name' is not defined

[User]
You are a strict SQL translator. 

You MUST ONLY output a single SQL query. 

You are NOT allowed to repeat the instructions, the examples, or any natural language text. 

You CANNOT explain anything. 

You MUST copy the style of the EXAMPLES exactly.

Respond ONLY with the SQL query.

---

[EXAMPLE NL QUESTION 1] i am interested in a flight on american airlines from dallas to washington
[EXAMPLE SQL PROMPT 1] SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 WHERE flight_1.airline_code = 'AA' AND ( flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DALLAS' AND flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'WASHINGTON' )

[EXAMPLE NL QUESTION 2

In [10]:
df_test_in.iloc[0,0]

'please list the flight times from newark to boston'

In [ ]:
len_sql_list = [len(s.split()) for s in atis_df.sql_query]

import numpy as np
k = 25
tops_ind = np.argpartition(len_sql_list, k)[-k:]
for ind in tops_ind:
    print(len_sql_list[ind], atis_df.sql_query[ind])

86 SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.departure_time BETWEEN 0 AND 800 AND ( flight_1.stops = 0 AND ( flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'NEW YORK' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'LAS VEGAS' AND flight_1.flight_days = days_1.days_code AND days_1.day_name = date_day_1.day_name AND date_day_1.year = 1991 AND date_day_1.month_number = 2 AND date_day_1.day_number = 28 ) ) )


In [14]:
from database import run_sql_comparison_pipeline


# Replace these with your actual queries
query1 = "SELECT DISTINCT flight_id FROM flight LIMIT 10;"
query2 = "SELECT DISTINCT flight_id FROM flight LIMIT 10;"
    
run_sql_comparison_pipeline(query1, query2)

ImportError: cannot import name 'run_sql_comparison_pipeline' from 'database' (/home/prompt5398/LLM/src/database.py)

In [15]:
from database import *